In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys
sys.path.append("/home/files/feature_orthogonality")
from utils.session_config import setup_gpus

In [ ]:
import tensorflow as tf
keras = tf.keras
import cv2
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
setup_gpus(memory_fraction=0.2)

In [ ]:
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPool2D, Flatten, Dense, ReLU, BatchNormalization, GlobalAveragePooling2D
from utils.losses import L2_Orthogonal
from utils.BEP import convert_to_separate_perceptrons, Back_etching_propagation
from utils.layers import Etching_Layer

In [ ]:
trained_model = tf.keras.models.load_model("mnist-3-cat-classifier.h5")

In [ ]:
bep_algorithm = Back_etching_propagation(trained_model)

In [ ]:
# model = bep_algorithm.return_model()
# model.summary()
#model.output

# Data acquisition

In [ ]:
activations = []
input_layer = trained_model.input
x = input_layer
for layer in trained_model.layers:  
    if layer.__class__.__name__ == 'ReLU':
        x = layer(x)
        o = GlobalAveragePooling2D()(x)
        activations.append(o)
    else:
        x = layer(x)
model = tf.keras.Model(input_layer, outputs=[x, activations])

In [ ]:
model.output[1]

In [ ]:
import pandas as pd
L1 = pd.DataFrame(columns=list(range(16))+['class'])
L2 = pd.DataFrame(columns=list(range(16))+['class'])
L3 = pd.DataFrame(columns=list(range(16))+['class'])

In [ ]:
L1

In [ ]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()
train_x = train_x[:,:,:,tf.newaxis]/255.
test_x = test_x[:,:,:,tf.newaxis]/255.
print(train_x.shape)
print(test_x.shape)

In [ ]:
x,o = model(train_x)

In [ ]:
train_y.shape

In [ ]:
k = o[0].numpy()
for i in range(6000):
    L1.loc[i] = list(k[i]) + [train_y[i]]

In [ ]:
k = o[1].numpy()
for i in range(6000):
    L2.loc[i] = list(k[i]) + [train_y[i]]

In [ ]:
k = o[2].numpy()
for i in range(6000):
    L3.loc[i] = list(k[i]) + [train_y[i]]

In [ ]:
def convert2int(x):
    return int(x)
L1['class'] = L1['class'].apply(convert2int)
L2['class'] = L2['class'].apply(convert2int)
L3['class'] = L3['class'].apply(convert2int)

In [ ]:
L3.head(25)

In [ ]:
L1.groupby('class').mean()

In [ ]:
L2.groupby('class').mean()

In [ ]:
L3.groupby('class').mean()

In [ ]:
L3.groupby('class').mean().iloc[[0,5,7]]

In [ ]:
L3.groupby('class').mean().iloc[:,13:]

In [ ]:
L3.groupby('class').mean().iloc[['0', '5', '7']]

In [ ]:
L3.groupby('class').mean().iloc[[1,2,3,4,6,8,9]]